In [1]:
VERSION = '99_v1_02_06'

In [2]:
model_list = [    
    {'path':'../../13_Baseline5/exp/result/13_v1_03/oof_df.csv', 'w':0.5}, #deberta-v3-base, seed100, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_04/oof_df.csv', 'w':1}, #deberta-v3-base, seed200, split00_v2_07
    {'path':'../../13_Baseline5/exp/result/13_v1_07/oof_df.csv', 'w':0.5}, #deberta-base, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_03/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_04/oof_df.csv', 'w':0.5}, #longformer-large, seed100, split00_v2_07
]

In [3]:
import pandas as pd

w_list = []
for i,model in enumerate(model_list):
    w = model['w']
    w_list.append(w)

stack_df = None
for i,model in enumerate(model_list):
    pred_df = pd.read_csv(model['path'])
    w = w_list[i]
    pred_df['pred_ineffective'] = pred_df['pred_ineffective'] * w / sum(w_list)
    pred_df['pred_adequate'] = pred_df['pred_adequate'] * w / sum(w_list)
    pred_df['pred_effective'] = pred_df['pred_effective'] * w / sum(w_list)
    pred_df = pred_df.rename(columns={
        'pred_ineffective':f'Ineffective_{i}', 
        'pred_adequate':f'Adequate_{i}', 
        'pred_effective':f'Effective_{i}'
    })
    if stack_df is None:
        stack_df = pred_df.copy()
    else:
        stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                  on='discourse_id', how='left')
    

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
oof_df = train_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

In [4]:
for col_name in ['Ineffective','Adequate','Effective']:
    cols = [f'{col_name}_{i}' for i in range(len(model_list))]
    oof_df[col_name] = oof_df[cols].sum(axis=1)

In [5]:
from sklearn.metrics import log_loss

score = log_loss(oof_df['label'].values, oof_df[['Ineffective','Adequate','Effective']].values)
print('oof={:.4f}'.format(score))

oof=0.5904


In [6]:
import os
os.makedirs('result', exist_ok=True)

oof_df.to_csv(f'result/stack_{VERSION}.csv', index=False)

In [7]:
oof_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Ineffective_0,Adequate_0,Effective_0,label,loss,...,Effective_2,Ineffective_3,Adequate_3,Effective_3,Ineffective_4,Adequate_4,Effective_4,Ineffective,Adequate,Effective
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0.004596,0.128424,0.009837,1,0.106507,...,0.019735,0.009225,0.264238,0.012251,0.001000,0.116014,0.025843,0.028764,0.899689,0.071547
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0.014678,0.126776,0.001403,1,0.119425,...,0.007895,0.014992,0.263233,0.007489,0.010160,0.126896,0.005801,0.063872,0.911001,0.025126
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0.039009,0.103240,0.000608,1,0.324790,...,0.003183,0.072405,0.210553,0.002757,0.010405,0.127156,0.005296,0.192049,0.795053,0.012898
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0.029824,0.111454,0.001580,1,0.248233,...,0.002466,0.068198,0.214155,0.003361,0.036436,0.102802,0.003619,0.258172,0.729179,0.012649
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0.044976,0.097321,0.000560,1,0.383832,...,0.001205,0.098344,0.185625,0.001745,0.023646,0.115180,0.004031,0.297506,0.694321,0.008174
